# Performant Code 

## Learning Objectives 
- Understand basic strategies for writing high-performance Julia code 
- Learn about type stability and why it's important for performance 
- Recognise the impact of global variables on performance and how to avoid them 
- Appreciate the benefit of using built-in functions and vectorised operations for optimisation 
- Choose appropriate data structures for a task to improve performance 
- Understand memory management techniques (like avoiding unnecessary copies) to write efficient code 
- Measure running time and memory allocation of code and identify bottlenecks using simple tools 

One of Julia's major appeals is performance. You can often write code in Julia that is both high-level and also runs nearly as fast as lower-level languages. However, to fully unlock this performance, it's good to be aware of a few tips and practices. Within this lesson, we are going to introduce some key concepts: type stability, avoiding globals, using efficient approaches, and basic profiling/timing.

## Overview of Performance Tuning Strategies 
- Performance tuning in Julia often comes down to writing code that is easy for the compiler to optimise, which includes: 
- Ensuring computations are type-stable: the types of variables don't change unpredictably. 
- Avoiding global variables in tight loops or computations: use functions to encapsulate logic. 
- Utilising Julias vectorised (broadcast) operations and built-in functions, which are highly optimised, instead of manually looping in an inefficient way. 
- Choose the right data structure, e.g. using arrays, tuples, and dictionaries appropriately. 
- Reducing memory allocations when possible, for example, by modifying data in place or using views for subarrays instead of making copies. 
- Measuring and profiling to find where the time is actually being spent, so you can optimise where it matters.

## Efficient Julia Code 

### Type Stability 

A function is type-stable if the type of its output can be determined from the types of its inputs, **without having to run the function**. This means that you need to: 
- pass inputs of known types into the function, 
- the compiler can predict what **type** the result will be.

Example of a type-stable function: 

```Julia
function add(x::Float64, y::Float64)
    return x + y
end
```
For the function above:
- If you give two `Float64`s, the result is always a `Float64`.
- The compiler **knows this immediately**.

Example of a type-unstable function: 
```Julia
function maybe_add(x, y)
    if rand() > 0.5
        return x + y
    else
        return string(x, y)
    end
end
```
- Sometimes it returns a **number**, sometimes a **String**. 
- The compiler **Can't predict** the output type just from the input types. 
- This forces Julia to **insert expensive type checks** at runtime. 

This then raises the question of **Does specifying types in function signatures improve performance?** Where the answer is: No, Julia specialises in methods on the concrete types it actually sees, even without annotations. Type annotations in signatures mainly help with dispatch, readability, and documentation; they rarely affect raw speed except in rare corner cases, as discussed within the [Julia documentation](https://docs.julialang.org/en/v1/manual/functions/#Argument-type-declarations).

Specifying types such as: 

```Julia 
function myfun(x::Float64)
    ...
end
```

It will not automatically make the code faster. The reason for this is: 
Julia **compiles specialised versions of functions for the types it sees anyway**, even if you didn't specify types.
What matters for performance is **what happens inside the function** - are the types predictable?

**You can write a function with no type annotations** but still make it very fast if the output type is **predictable**. 

The rule of thumb is:
- Specifying types at the function input is good for readability, documentation, dispatch (method choices)
- Type stability inside the function is crucial for performance.

You can use: 
```Julia 
@code_warntype your_function(args)
```

**An aside on macros**: In Julia, **macros** are special functions that operate on your *code* itself, rather than on the *values* your code produces. They run at **compile** time, transforming expressions before any actual execution happens. That means you can **generate boilerplate** code automatically (e.g. logging) or **inject** or **wrap** user code with additional checks or timings. You invoke a macro by prefixing its name with `@` and then supplying the target expression (often a function call) in parentheses, as done above for `@code_warntype`.

To check if a function is type-stable. It will show **orange** (or **red**) types if Julia can not predict types.

#### Type-Stable Function: `add`

In [1]:
function add(x::Float64, y::Float64)
    return x + y
end

@code_warntype add(1.0, 2.0)

MethodInstance for add(::Float64, ::Float64)
  from add(x::Float64, y::Float64) @ Main In[1]:1
Arguments
  #self#::Core.Const(Main.add)
  x::Float64
  y::Float64
Body::Float64
1 ─ %1 = Main.:+::Core.Const(+)
│   %2 = (%1)(x, y)::Float64
└──      return %2



Above, you can see a type-stable function `add`. The output from `@code_warntype` shows that the output will be a `Float64`; all the variable types are clearly inferred, and there are no concerning red or yellow types. Given two `Float64` arguments, the output is preddictably a `Float64`. Julia can compile optimised machine code with no dynamic checks, which is the ideal case for performance. 

#### Mildly Type-Unstable Function: `maybe_divide`

In [2]:
function maybe_divide(x, y)
    x > y ? x / y : nothing
end

@code_warntype maybe_divide(4.0, 2.0)

MethodInstance for maybe_divide(::Float64, ::Float64)
  from maybe_divide(x, y) @ Main In[2]:1
Arguments
  #self#::Core.Const(Main.maybe_divide)
  x::Float64
  y::Float64
Body::Union{Nothing, Float64}
1 ─ %1 = Main.:>::Core.Const(>)
│   %2 = (%1)(x, y)::Bool
└──      goto #3 if not %2
2 ─ %4 = Main.:/::Core.Const(/)
│   %5 = (%4)(x, y)::Float64
└──      return %5
3 ─ %7 = Main.nothing::Core.Const(nothing)
└──      return %7



Above, you can see the mildly type-unstable function `maybe_divide`. In the `@code_warntype maybe_divide(4.0, 2.0)` output, the return slot is highlighted in yellow as `Body::Union{Nothing, Float64}`, indicating Julia infers a union of `Float64` and `Nothing`. Because `maybe_divide` may either compute `x` / `y` (yielding a `Float64`) or return `nothing`, the compiler can’t predict a single concrete return type. At runtime, Julia must handle both possibilities via branching and dynamic dispatch, incurring boxing and method‐lookup overhead that hurts performance compared to a fully type-stable function.

#### Severely Type-Unstable Function: `bad_sum`

In [3]:
function get_first(x)
    x[1]
end


@code_warntype get_first([1, 2.0, "3"])


MethodInstance for get_first(::Vector{Any})
  from get_first(x) @ Main In[3]:1
Arguments
  #self#::Core.Const(Main.get_first)
  x::Vector{Any}
Body::Any
1 ─ %1 = Base.getindex(x, 1)::Any
└──      return %1



Above, you can see a severely type-unstable function, `get_first(x)`. The crucial line is `Body::Any` highlighted in **red**, indicating Julia can only infer that the return value is of type Any. As the input vector is a `Vector{Any}`, Julia has no way to know at compile time what type `x[1]` will be. As a result, **dynamic dispatch occurs on every call, and the returned element is **boxed** as `Any`, resulting in a number of runtime type-checks and heap allocations. Boxing values means they are stored in the heap with additional type information rather than directly in registers. Together, this dramatically degrades performance when compared with using a concrete container (e.g. `Vector{Int}`), where the return type can be fully inferred, due to slower memory access, dynamic type checks, and an increased overhead of garbage collection. 

One thing of note is that if we had modified the call to `get_first`, to be:

In [4]:
@code_warntype get_first([1, 2, 3])

MethodInstance for get_first(::Vector{Int64})
  from get_first(x) @ Main In[3]:1
Arguments
  #self#::Core.Const(Main.get_first)
  x::Vector{Int64}
Body::Int64
1 ─ %1 = Base.getindex(x, 1)::Int64
└──      return %1



Here, `get_first` is still technically *type‐unstable* in the abstract sense, because its return type depends entirely on what you pass in, and the compiler can’t know ahead of time what `x[1]` will be. However, in practice, that doesn’t always translate to slow code. The key line, `Body::Int64`, shows **no warnings**: Julia knows that for a `Vector{Int64}`, the first element **must** be an `Int64`, so it emits specialised, fully‐typed code with zero dynamic checks.

The takeaway is that the *container’s* element type matters just as much as your function. A function like get_first can be “unstable” in theory, yet still perform optimally whenever you feed it a **concrete‐typed** collection (e.g. `Vector{Int64}`). By contrast, using abstract‐typed containers (e.g. `Vector{Any}`) forces Julia to fall back to dynamic dispatch and boxing. Keeping your collection parameterised over concrete element types is a crucial performance tip in Julia.

### Avoiding Global variables 

Julia compiles and optimises function just-in-time (JIT) when you call them, but code that lives in the global scope, such as a top-level loop that uses global variables, cannot be optimised as effectively, since globals might change type or value at any moment. 

**Best Practice**: encapsulate all performance-critical work inside a function, then invoke those functions from the global scope. By doing so, you ensure the compiler sees only local variables with known, concrete types. 

For instance, rather than writing at the top level: 

```julia 
# Global loop – avoids type instability but still limits optimisation
numbers = rand(1000)
total = 0.0
for x in numbers
    total += x
end
total
```

You would instead want to define and call a function: 

```julia 
function sum_array(arr)
    total = 0.0            # local Float64
    for x in arr           # x is Float64
        total += x
    end
    return total
end

numbers = rand(1000)
total = sum_array(numbers)
```

Inside `sum_array`, both `total` and `x` have fixed, known types, allowing the compiler to produce highly optimised code. In the global version, even though `total` is initialised as a `Float64`, its status as a global variable prevents the same level of optimisation. 

You *can* annotate a globals type:

```julia
global_total::Float64 = 0.0
```

To forbid it from ever changing to another type, but this still won't unlock all the optimisations you get inside functions. This is discussed further in the [Julia documentation](https://docs.julialang.org/en/v1/manual/performance-tips/#Avoid-untyped-global-variables).

For truly constant values, always declare them with `const` at the global level. 

```julia 
const PI = 3.14
```

This signals to the compiler that `PI` will never change, enabling further speedups. 



## Measuring Performance 

Julia provides some simple macros to measure execution time and memory: 
- `@time expression` runs `expression` once, printing both execution time (including compilation on the first run) and memory allocated. To see the "steady-state" performance (without compile-time overhead), run it a second time. 
- `@benchmark expression` (from `BenchmarkTools.jl`) runs `expression` many times, automatically "warming-up" the function. It reports statistics (min, median, mean, allocations, etc), so you avoid compile-time bias and see true variability. 
- `@timed expression` returns a `Timing` object containing fields like `time` and `allocs` rather than printing them, so you can programmatically inspect or log those metrics. 
- `@allocated expression` returns just the number of bytes allocated by running `expression`.

In [5]:
function method_0(N)
   arr = Int[]
   for i in 1:N
       arr = vcat(arr, Int[i])
   end
   return sum(arr)
end

function method_1(N)
    arr = Int[]  
    for i in 1:N
        push!(arr, i)      
    end
    return sum(arr)        
end

println("Method_0 — first run:")
@time method_0(100_000)

println("Method_0 — second run:")
@time method_0(100_000)

println("Method_1 — first run:")
@time method_1(100_000)

println("Method_1 — second run:")
@time method_1(100_000)

println("Completed")

Method_0 — first run:
  2.223694 seconds (499.75 k allocations: 38.744 GiB, 24.08% gc time)
Method_0 — second run:
  2.480468 seconds (499.75 k allocations: 38.744 GiB, 25.48% gc time)
Method_1 — first run:
  0.000237 seconds (18 allocations: 1.928 MiB)
Method_1 — second run:
  0.000172 seconds (18 allocations: 1.928 MiB)
Completed


`method_0` rebuilds the array with `vcat` on every iteration, causing ~500 000 separate heap allocations and copying 38.7 GiB of data. This allocation and garbage-collection overhead dominates its ≈2.2 s runtime on both runs. `method_1` uses `push!` to grow the array incrementally. Thanks to Julia’s amortised resizing strategy, it only incurs 18 allocations totalling ~1.9 Mib, so it finishes in under 0.2 ms. Running `@time` twice shows that **both methods** pay the JIT compilation cost on their first call, but only `method_0` continues to pay massive allocation and GC costs thereafter, whereas `method_1` remains lightning-fast with negligible heap traffic.

### Profiling for Bottlenecks 

If you have a complex program and you want to see where it spends time, you can use the built-in profiler: 

```Julia 
using Profile
@profiler my_long_running_function()
```

Then use `Profile.print()` or the `ProfileView.jl` package to analyse the results. Profiling tells you which functions or lines are taking the most time. For simpler cases, you might not need this level of detail.

## Vectorised (Broadcast) Operations

Julia's "vectorised" operation, more accurately, **broadcasted** operations, uses the dot (`.`) syntax to apply an element-wise transformation without writing an explicit loop. Under the hood, broadcasting compiles down to a loop, so there is **no inherent speed advantage** over a well-written `for` loop once both are compiled. For example:

```julia
julia> function plus1(arr)
           result = similar(arr)
           for i in eachindex(arr)
               result[i] = 1 + arr[i]
           end
           return result
       end
plus1 (generic function with 1 method)

julia> plus1_vec(arr) = 1 .+ arr
plus1_vec (generic function with 1 method)

julia> data = rand(1_000_000);

julia> @time plus1(data);
  0.010614 seconds (4.84 k allocations: 7.962 MiB, 78.82% compilation time)

julia> @time plus1(data);  # compilation not included
  0.002725 seconds (2 allocations: 7.629 MiB)

julia> @time plus1_vec(data);
  0.045433 seconds (115.61 k allocations: 15.702 MiB, 95.24% compilation time)

julia> @time plus1_vec(data);  # compilation not included
  0.002058 seconds (2 allocations: 7.629 MiB)
```

The **first run** of each function shows significant compilation overhead (high `% compilation time`). The **second run** (after JIT compilation) demonstrates nearly identical execution time (~2ms) and allocations for both `plus1` and `plus1_vec`. This means that `broadcast syntax` (`1 .+ arr`) is primarily syntactic sugar for a loop; it doesn't magically beat an explicit `for` loop. 

### Utilising Built-In Functions
Julia's standard library and well-known packages have many optimised routines, which are often implemented in C or using Julia's own optimisations (including multi-threading in some cases). Examples: `sum`, `maximum`, or linear algebra operations (`A * B`) or sorting (`sort`). 

#### Using Appropriate Data Structures 
Some key considerations when determining which data structure to use include: 
- If you need random access to elements by index and the collection will grow/shrink, you require a Vector (`Array`). 
- If you need to look up values by keys, use a `Dict` instead of searching through an array each time. 
- If you have a fixed small set of values of heterogenous types, a `Tuple` can be helpful. They are immutable, and their types are part of their identity, making them very efficient for specific uses, like returning multiple values from a function. 
- If you need stack or queue behaviour, you can still use arrays (with `push!` or `pop!` for the stack and `push!` and `popfirst!` for the queue. 
- If you have binary data or bits, consider `BitVector` for large boolean arrays that are memory efficient. 
- For mathematical operations, using native numeric types (`Int`, `Float64`) is faster than arbitrary precisions or rational types, so only use `BigInt`, `BigFloat` and `Rational` when needed. 

#### Memory Management and Avoiding Unnecessary Allocation 

In Julia, **memory allocation** of interest refers primarily to **heap allocation**, when your code requests new blocks of memory at runtime. Each allocation incurs overhead (for bookkeeping) and later increases garbage-collection work, which can slow down loops or large-scale data processing. 

Common sources of excessive allocation include **Type-stability issues**, which force Julia to box values on the heap, and **creating many small temporary arrays**, for example via repeated slicing or non-in-place broadcast, as discussed in the [Julia documentation](https://docs.julialang.org/en/v1/manual/performance-tips/#Measure-performance-with-%5B@time%5D(@ref)-and-pay-attention-to-memory-allocation). 

Some strategies to cut down heap usage include: 
**Pre-allocate and reuse arrays** instead of building a new array each iteration:

```julia
# Bad: allocates a new array inside the loop
results = []
for i in 1:1000
    push!(results, compute(i))
end

# Good: allocate once, then fill in place
results = Vector{Float64}(undef, 1000)
for i in 1:1000
    results[i] = compute(i)
end
```

**Use views for subarrays** `view(A, 1:10)` creates a lightweight window into `A` without copying its data, avoiding a fresh allocation. 

**Favour in-place operations** Functions ending in `!` typically modify their arguments rather than returning new objects. 

```julia 
sort!(array)    # rearranges in place
push!(arr, x)   # grows the same array
```

**Avoid repeated type conversions inside loops**. Converting values one by one triggers allocations or passing work on each iteration.

```julia
# Bad: parse inside the loop, resulting in repeated allocations
for s in string_numbers
    x = parse(Int, s)
    process(x)
end

# Good: convert once, then loop
ints = parse.(Int, string_numbers)
for x in ints
    process(x)
end
```

**Tip**: Use `@time` or @BenchmarkTools.@btime` and watch the "allocations" count. You will often spot allocations caused by temporary arrays or type-unstable code. 

##### Clever Algorithms / Calculations 

Often, the most considerable speedups come not from micro-optimising memory, but from ROM choosing better algorithms or mathematical shortcuts. For example, instead of summing the first *N* integers in a loop: 

```julia 
function sum1(N)
    total = 0
    for i in 1:N
        total += i
    end
    return total
end
```

You can use the closed-form formula: 


```julia 
sum2(N) = N * (N + 1) ÷ 2
```

Which runs in constant time rather than being based on the size of the input, and allocates no extra memory. Whenever possible, look for algorithmic improvements or analytic formulas before resorting to low-level optimisations. 

## Exercise: Analysing the Performance of Code 

Given the three functions below, use what we've discussed so far about type stability, allocations, and performance to understand **why they perform differently**.

```Julia 
function method_1(N)
    arr = Int[]  
    for i in 1:N
        push!(arr, i)      
    end
    return sum(arr)        
end

function method_2(N)
    arr = collect(1:N)    
    return sum(arr)
end


function method_3(N)
    return N*(N+1)÷2
end
```

# End of Section Quiz

In [6]:
using JSON

function show_quiz_from_json(path)
    quiz_data = JSON.parsefile(path)

    html = """
    <style>
    .quiz-question {
        background-color: #6c63ff;
        color: white;
        padding: 12px;
        border-radius: 10px;
        font-weight: bold;
        font-size: 1.2em;
        margin-bottom: 10px;
    }

    .quiz-form {
        margin-bottom: 20px;
    }

    .quiz-answer {
        display: block;
        background-color: #f2f2f2;
        border: none;
        border-radius: 10px;
        padding: 10px;
        margin: 5px 0;
        font-size: 1em;
        cursor: pointer;
        text-align: left;
        transition: background-color 0.3s;
        width: 100%;
    }

    .quiz-answer:hover {
        background-color: #e0e0e0;
    }

    .correct {
        background-color: #4CAF50 !important;
        color: white !important;
        border: none;
    }

    .incorrect {
        background-color: #D32F2F !important;
        color: white !important;
        border: none;
    }

    .feedback {
        margin-top: 10px;
        font-weight: bold;
        font-size: 1em;
    }
    </style>

    <script>
    function handleAnswer(qid, aid, feedback, isCorrect) {
        // Reset all buttons for the question
        let buttons = document.querySelectorAll(".answer-" + qid);
        buttons.forEach(btn => {
            btn.classList.remove('correct', 'incorrect');
        });

        // Apply correct/incorrect to selected
        let selected = document.getElementById(aid);
        selected.classList.add(isCorrect ? 'correct' : 'incorrect');

        // Show feedback below the question
        let feedbackBox = document.getElementById('feedback_' + qid);
        feedbackBox.innerHTML = feedback;
        feedbackBox.style.color = isCorrect ? 'green' : 'red';
    }
    </script>
    """

    for (i, question) in enumerate(quiz_data)
        qid = "$i"
        html *= """<div class="quiz-question">$(question["question"])</div><form class="quiz-form">"""

        for (j, answer) in enumerate(question["answers"])
            aid = "q$(i)_a$(j)"
            feedback = answer["feedback"]
            correct = startswith(lowercase(feedback), "correct")
            html *= """
            <button type="button" class="quiz-answer answer-$qid" id="$aid"
                onclick="handleAnswer('$qid', '$aid', '$feedback', $(correct))">
                $(answer["answer"])
            </button>
            """
        end

        html *= """<div class="feedback" id="feedback_$qid"></div></form><hr>"""
    end

    display("text/html", html)
end


# Use the function
show_quiz_from_json("questions/summary_performant_code.json")

Which function is recommended for pre-allocating an array of zeros in Julia for better performance? 
 array_zeros()
 
 
 zeros()
 
 
 preallocate()
 
 
 fill(0, dims...)
 
 In Julia, why should you avoid growing arrays inside a loop without pre-allocation? 
 It uses too much memory instantly
 
 
 It causes syntax errors
 
 
 It results in repeated memory reallocation, which slows down the code
 
 
 It leads to incorrect computation results
 
 What is a general tip for writing more performant Julia code when using loops? 
 Always use global variables inside loops
 
 
 Minimise memory allocation inside loops
 
 
 Nest as many loops as possible
 
 
 Prefer string operations over numerical operations